## Crop the SWE rasters to the NVASA basins

In [4]:
import subprocess
import geopandas as gpd

def fix_geometry(df):
    if df.geometry.is_valid: # if the geometry is valid in shapely, do nothing
        return df.geometry
    
    elif df.geometry.is_valid==False: # if the geometry is invalid, fix it with a buffer of 0
        return df.geometry.buffer(0)

In [7]:
# clean the GageBains file
dat = gpd.read_file('./data/GageBasins.shp')
dat['new_geom'] = dat.apply(fix_geometry,axis=1)
dat = dat.set_geometry('new_geom')
del dat['geometry']

dat.to_file('./data/GageBasins_clean.shp')

In [8]:
df = gpd.read_file('./data/GageBasins_clean.shp')

In [30]:
names = df.Name.unique()
wateryears = range(2004,2017)
shp = './data/GageBasins_clean.shp'

for wy in wateryears:
    for name in names:
        
        # crop the peak SWE
        infl = './data/snodas_peakSWE_wy%i.tiff'%wy
        outfl = './data/snodas_cropped/peakSWE_wy%i_%s.tiff'%(wy,name)
        
        cmd = "gdalwarp -overwrite -cutline %s -cwhere "'"Name='"'%s'"'"'" -crop_to_cutline %s %s"%(shp,name,infl,outfl)
        subprocess.call(cmd,shell = True)
        
        # crop the DOWY peak SWE
        infl = './data/snodas_DOWYpeakSWE_wy%i.tiff'%wy
        outfl = './data/snodas_cropped/DOWYpeakSWE_wy%i_%s.tiff'%(wy,name)
        
        cmd = "gdalwarp -overwrite -cutline %s -cwhere "'"Name='"'%s'"'"'" -crop_to_cutline %s %s"%(shp,name,infl,outfl)
        subprocess.call(cmd,shell = True)
        
        # crop the DOWY no snow
        infl = './data/snodas_DOWYnoSnow_wy%i.tiff'%wy
        outfl = './data/snodas_cropped/DOWYNS_wy%i_%s.tiff'%(wy,name)
        
        cmd = "gdalwarp -overwrite -cutline %s -cwhere "'"Name='"'%s'"'"'" -crop_to_cutline %s %s"%(shp,name,infl,outfl)
        subprocess.call(cmd,shell = True)